In [11]:
#File: dataPreprocessing.ipynb
#Purpose: data preprocessing for steve mapping dataset
#Author: Quan Gan
import pandas as pd
import numpy as np

In [12]:
df = pd.read_csv('steve_mapping_1000.csv', skiprows = 1)

In [13]:
info = df[['locality','higherClassification', 'Steve label']]
count = 0
for index,row in info.iteritems():
    #recrods' labels clean
    info = info.replace(np.nan, "", regex=True)
    info = info[info['Steve label'] != ""]
    info = info.replace('\n', " ", regex=True)
    info = info.replace('Lake,', "Lake", regex=True)
    
    #add prefix "__label__" to labels
    labels = info['Steve label'].str.split(r', ', expand=True)
    labels = labels.rename(columns= {0: "label1", 1: "label2"})
    labels['label1'] = "__label__" + labels['label1']
    labels = labels.fillna("")
    labels.loc[labels['label2'] != "", 'label2'] = "__label__"+labels.loc[labels['label2'] != ""]['label2'] 
    
    result = labels.replace(" ", "_", regex=True)
    
    info['text'] = np.array([ '' for i in range(995)])
    #records' text clean
    for new_index, new_row in info.iteritems():
        info['text'] += info[new_index] + " "
    
    info['text'] = info['text'].str.replace('[^\w\s]','',regex = True)
    info['text'] = info['text'].str.lower()
    
    info = info.drop(columns = [index])
    
    #sample records to trainset and testset
    result['label'] = result['label1'] + " " + result['label2']
    result['text'] = info['text']
    ds = result['label'] + " " + result['text']
    train = ds.sample(frac=0.7, random_state=99) #randomly choose 70% records as trainset
    test = ds.loc[~ds.index.isin(train.index)] #rest 30% records as testset
    
    name = ['full', 'locality','higherClassification']
    
    ds.to_csv(r'cleanedData' + name[count] + "_remove" + '.txt', header=False, index=False)
    train.to_csv(r'steve_696.train', header=False, index=False)
    test.to_csv(r'steve_299.valid', header=False, index=False)   
    
    count += 1